In [1]:
import re
import yaml
from pathlib import Path
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv
from crewai import LLM

load_dotenv()

True

In [2]:
llm = LLM(
    model="openai/gpt-4", # call model by provider/model_name
    temperature=0.8,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.1,
    presence_penalty=0.1,
    stop=["END"],
    seed=42
)

In [3]:

# Use Path for file locations
current_dir = Path.cwd()
agents_config_path = current_dir / "config" / "agents.yaml"
tasks_config_path = current_dir / "config" / "tasks.yaml"

# Load YAML configuration files
with open(agents_config_path, "r") as file:
    agents_config = yaml.safe_load(file)

with open(tasks_config_path, "r") as file:
    tasks_config = yaml.safe_load(file)

## Define Agents
spamfilter = Agent(
    llm=llm,
    config=agents_config["spamfilter"], allow_delegation=False, verbose=True
)

analyst = Agent(config=agents_config["analyst"], allow_delegation=False, verbose=True)

scriptwriter = Agent(
    llm=llm,
    config=agents_config["scriptwriter"], allow_delegation=False, verbose=True
)

formatter = Agent(
    llm=llm,
    config=agents_config["formatter"], allow_delegation=False, verbose=True
)


scorer = Agent(config=agents_config["scorer"], allow_delegation=False, verbose=True)

In [4]:
discussion = """From: keith@cco.caltech.edu (Keith Allan Schneider)
Subject: Re: <Political Atheists?
Organization: California Institute of Technology, Pasadena
Lines: 50
NNTP-Posting-Host: punisher.caltech.edu

bobbe@vice.ICO.TEK.COM (Robert Beauchaine) writes:

>>I think that about 70% (or so) people approve of the
>>death penalty, even realizing all of its shortcomings.  Doesn't this make
>>it reasonable?  Or are *you* the sole judge of reasonability?
>Aside from revenge, what merits do you find in capital punishment?

Are we talking about me, or the majority of the people that support it?
Anyway, I think that "revenge" or "fairness" is why most people are in
favor of the punishment.  If a murderer is going to be punished, people
that think that he should "get what he deserves."  Most people wouldn't
think it would be fair for the murderer to live, while his victim died.

>Revenge?  Petty and pathetic.

Perhaps you think that it is petty and pathetic, but your views are in the
minority.

>We have a local televised hot topic talk show that very recently
>did a segment on capital punishment.  Each and every advocate of
>the use of this portion of our system of "jurisprudence" cited the
>main reason for supporting it:  "That bastard deserved it".  True
>human compassion, forgiveness, and sympathy.

Where are we required to have compassion, forgiveness, and sympathy?  If
someone wrongs me, I will take great lengths to make sure that his advantage
is removed, or a similar situation is forced upon him.  If someone kills
another, then we can apply the golden rule and kill this person in turn.
Is not our entire moral system based on such a concept?

Or, are you stating that human life is sacred, somehow, and that it should
never be violated?  This would sound like some sort of religious view.
 
>>I mean, how reasonable is imprisonment, really, when you think about it?
>>Sure, the person could be released if found innocent, but you still
>>can't undo the imiprisonment that was served.  Perhaps we shouldn't
>>imprision people if we could watch them closely instead.  The cost would
>>probably be similar, especially if we just implanted some sort of
>>electronic device.
>Would you rather be alive in prison or dead in the chair?  

Once a criminal has committed a murder, his desires are irrelevant.

And, you still have not answered my question.  If you are concerned about
the death penalty due to the possibility of the execution of an innocent,
then why isn't this same concern shared with imprisonment.  Shouldn't we,
by your logic, administer as minimum as punishment as possible, to avoid
violating the liberty or happiness of an innocent person?

keith
"""

In [5]:
# Filter out spam and vulgar posts
task0 = Task(
    description=tasks_config["task0"]["description"],
    expected_output=tasks_config["task0"]["expected_output"],
    agent=spamfilter,
)

# process post with a crew of agents, ultimately delivering a well formatted dialogue
task1 = Task(
    description=tasks_config["task1"]["description"],
    expected_output=tasks_config["task1"]["expected_output"],
    agent=analyst,
)

task2 = Task(
    description=tasks_config["task2"]["description"],
    expected_output=tasks_config["task2"]["expected_output"],
    agent=scriptwriter,
)

task3 = Task(
    description=tasks_config["task3"]["description"],
    expected_output=tasks_config["task3"]["expected_output"],
    agent=formatter,
)
crew = Crew(
    agents=[analyst, scriptwriter, formatter],
    tasks=[task1, task2, task3],
    verbose=True,  # Crew verbose more will let you know what tasks are being worked on, you can set it to 1 or 2 to different logging levels
    process=Process.sequential,  # Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

In [6]:


result = crew.kickoff(inputs={  "discussion": discussion })

# get rid of directions and actions between brackets, eg: (smiling)
script = re.sub(r"\(.*?\)", "", result.raw)

print("===================== end result from crew ===================================")
print(script)
print("===================== score ==================================================")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 46a4d901-83b1-424f-a211-e6b1787b8a41                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: analyse                                                                                                 │
│                                                                                                                 │
│  Task: Analyse in much detail the following discussion: ### DISCUSSION: {From: keith@cco.caltech.edu (Keith     │
│  Allan Schneider)                                                                                               │
│  Subject: Re: <Political Atheists?                                                                              │
│  Organization: California Institute of Technology, Pasadena                                                     │
│  Lines: 50                                                                                                      │
│  NNTP-Posting-Host: punisher.caltech.edu                                                                        │
│                                                                                                                 │
│  bobbe@vice.ICO.TEK.COM (Robert Beauchaine) writes:                                                             │
│                                                                                                                 │
│  >>I think that about 70% (or so) people approve of the                                                         │
│  >>death penalty, even realizing all of its shortcomings.  Doesn't this make                                    │
│  >>it reasonable?  Or are *you* the sole judge of reasonability?                                                │
│  >Aside from revenge, what merits do you find in capital punishment?                                            │
│                                                                                                                 │
│  Are we talking about me, or the majority of the people that support it?                                        │
│  Anyway, I think that "revenge" or "fairness" is why most people are in                                         │
│  favor of the punishment.  If a murderer is going to be punished, people                                        │
│  that think that he should "get what he deserves."  Most people wouldn't                                        │
│  think it would be fair for the murderer to live, while his victim died.                                        │
│                                                                                                                 │
│  >Revenge?  Petty and pathetic.                                                                                 │
│                                                                                                                 │
│  Perhaps you think that it is petty and pathetic, but your views are in the                                     │
│  minority.                                                                                                      │
│                                                                                                                 │
│  >We have a local televised hot topic talk show that very recently                                              │
│  >did a segment on capital punishment.  Each and every advocate of                                              │
│  >the use of this portion of our system of "jurisprudence" cited the                                            │
│  >main reason for supporting it:  "That bastard deserved it".  True                                             │
│  >human compassion, forgiveness, and sympathy.                                                                  │
│                                                        

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: analyse                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Discussion Analysis                                                                                        │
│                                                                                                                 │
│  In this discussion, the main participants are Keith Schneider and Robert Beauchaine, discussing the topic of   │
│  capital punishment and its moral implications.                                                                 │
│                                                                                                                 │
│  1. **Robert Beauchaine's Position**:                                                                           │
│     - **Public Sentiment on Capital Punishment**: Beauchaine states that around 70% of people approve of the    │
│  death penalty, suggesting that this majority opinion lends it a level of reasonableness.                       │
│     - **Motivation for Support**: He attributes the support for the death penalty primarily to feelings of      │
│  revenge and fairness, arguing that many believe that a murderer deserves punishment equal to the crime they    │
│  committed. He emphasizes that it seems unfair for a murderer to continue living while their victim has died.   │
│                                                                                                                 │
│  2. **Keith Schneider's Challenge**:                                                                            │
│     - **Critique of Revenge**: Schneider critiques the notion of revenge as being "petty and pathetic,"         │
│  suggesting that such sentiments shouldn’t be the foundation for supporting capital punishment. He implies      │
│  that this view is not widely held, indicating he believes it's a minority opinion.                             │
│     - **Public Debate**: He references a local talk show segment where advocates of capital punishment          │
│  justified their position mainly on the basis that the murderer "deserved" death, critiquing this rationale     │
│  and associating it with a lack of true human compassion and empathy.                                           │
│                                                                                                                 │
│  3. **The Question of Moral Responsibility**:                                                                   │
│     - Schneider challenges the assumption that one must have compassion in cases of serious wrongdoing. He      │
│  suggests that the moral framework may actually support retaliatory justice, questioning whether societal       │
│  morals are based on a cycle of revenge. He posits that if one believes in such a framework, it is natural to   │
│  kill in response to a murder, drawing a parallel to the golden rule.                                           │
│                                                                                                                 │
│  4. **Fundamental Philosophical Questioning**:                                                                  │
│     - Schneider asks whether human life is sacred and questions if the value of life is inherently tied to      │
│  religious perspectives. He implies that holding life sacred could be a more profound reason to oppose capital  │
│  punishment.                                           

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 00a4da51-b5fd-4197-953c-fe21d848f32e                                                                     │
│  Agent: analyse                                                                                                 │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: scriptwriter                                                                                            │
│                                                                                                                 │
│  Task: Create a dialogue heavy screenplay from the discussion, between two persons. Do NOT write                │
│  parentheticals. Leave out wrylies. You MUST SKIP directional notes.                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: formatter                                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ## ROBERT BEAUCHAINE:                                                                                          │
│    The majority approve of the death penalty, Keith. About 70%. It's hard to argue against such a substantial   │
│  majority.                                                                                                      │
│                                                                                                                 │
│    ## KEITH SCHNEIDER:                                                                                          │
│    Just because it's a popular opinion, Robert, doesn't make it right. And revenge, which is what this really   │
│  comes down to, is petty and pathetic.                                                                          │
│                                                                                                                 │
│    ## ROBERT BEAUCHAINE:                                                                                        │
│    It's about fairness, Keith. A life for a life. How is it fair that a murderer gets to live when their        │
│  victim does not?                                                                                               │
│                                                                                                                 │
│    ## KEITH SCHNEIDER:                                                                                          │
│    I heard that argument on a talk show the                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: dffbdb42-1c2c-4f67-9f08-0820697d49f1                                                                     │
│  Agent: formatter                                                                                               │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 46a4d901-83b1-424f-a211-e6b1787b8a41                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ## ROBERT BEAUCHAINE:                                                                            │
│    The majority approve of the death penalty, Keith. About 70%. It's hard to argue against such a substantial   │
│  majority.                                                                                                      │
│                                                                                                                 │
│    ## KEITH SCHNEIDER:                                                                                          │
│    Just because it's a popular opinion, Robert, doesn't make it right. And revenge, which is what this really   │
│  comes down to, is petty and pathetic.                                                                          │
│                                                                                                                 │
│    ## ROBERT BEAUCHAINE:                                                                                        │
│    It's about fairness, Keith. A life for a life. How is it fair that a murderer gets to live when their        │
│  victim does not?                                                                                               │
│                                                                                                                 │
│    ## KEITH SCHNEIDER:                                                                                          │
│    I heard that argument on a talk show the                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: c74412be-5598-405a-bd0e-041783816c7a                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/c74412be-5598-405a-bd0e-041783816c7a?access_code=TRA │
│ CE-abb8c90132                                                                                                   │
│ 🔑 Access Code: TRACE-abb8c90132                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

===================== end result from crew ===================================
## ROBERT BEAUCHAINE:
  The majority approve of the death penalty, Keith. About 70%. It's hard to argue against such a substantial majority.
         
  ## KEITH SCHNEIDER:
  Just because it's a popular opinion, Robert, doesn't make it right. And revenge, which is what this really comes down to, is petty and pathetic.
         
  ## ROBERT BEAUCHAINE:
  It's about fairness, Keith. A life for a life. How is it fair that a murderer gets to live when their victim does not?
         
  ## KEITH SCHNEIDER:
  I heard that argument on a talk show the
===================== score ==================================================


In [7]:
task4 = Task(
    description=tasks_config["task4"]["description"],
    expected_output=tasks_config["task4"]["expected_output"],
    agent=scorer,
)

In [8]:
scorer.execute_task(
    task=task4,
    context=f"script: = {script}")

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: scorer                                                                                                  │
│                                                                                                                 │
│  Task: Score the following script: ### SCRIPT: {{script}}                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\m.lippolis\Desktop\multi agent\.venv\Lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Trace batch initialization returned status 401. Continuing without tracing.

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: scorer                                                                                                  │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  After assessing the dialogue based on several criteria, I would score it a 7 out of 10.                        │
│                                                                                                                 │
│  **Clarity:** The conversation is mostly clear, with participants expressing their viewpoints effectively.      │
│  However, some jargon around the death penalty could confuse a general audience.                                │
│                                                                                                                 │
│  **Relevance:** All points made are relevant to the topic of the death penalty, and both participants stay on   │
│  track.                                                                                                         │
│                                                                                                                 │
│  **Conciseness:** The dialogue is concise and direct, without unnecessary elaboration, which aids in            │
│  maintaining focus on the argument.                                                                             │
│                                                                                                                 │
│  **Politeness:** Both Robert and Keith maintain a level of respect, although there are undertones of            │
│  confrontation.                                                                                                 │
│                                                                                                                 │
│  **Engagement:** The exchange indicates that both parties are engaged and contribute their opinions             │
│  meaningfully, demonstrating interest in the topic.                                                             │
│                                                                                                                 │
│  **Flow:** The conversation flows logically from one point to the next, with a natural progression of ideas.    │
│  There are no awkward pauses or interruptions noted in the excerpt provided.                                    │
│                                                                                                                 │
│  **Coherence:** The dialogue maintains coherence, with both participants building on each other's arguments.    │
│                                                                                                                 │
│  **Responsiveness:** Each participant addresses the other's points adequately, showcasing a back-and-forth      │
│  exchange typical of a debate.                                                                                  │
│                                                                                                                 │
│  **Language Use:** The language used is appropriate for the context, with a mix of rhetorical questions and     │
│  ethical reasoning, although simplification might benefit accessibility.                                        │
│                                                                                                                 │
│  **Emotional Intelligence:** The emotional tone of the argument suggests a level of sensitivity regarding the   │
│  weight of the topic, although more empathy could enhan

"After assessing the dialogue based on several criteria, I would score it a 7 out of 10. \n\n**Clarity:** The conversation is mostly clear, with participants expressing their viewpoints effectively. However, some jargon around the death penalty could confuse a general audience. \n\n**Relevance:** All points made are relevant to the topic of the death penalty, and both participants stay on track.\n\n**Conciseness:** The dialogue is concise and direct, without unnecessary elaboration, which aids in maintaining focus on the argument.\n\n**Politeness:** Both Robert and Keith maintain a level of respect, although there are undertones of confrontation. \n\n**Engagement:** The exchange indicates that both parties are engaged and contribute their opinions meaningfully, demonstrating interest in the topic.\n\n**Flow:** The conversation flows logically from one point to the next, with a natural progression of ideas. There are no awkward pauses or interruptions noted in the excerpt provided.\n\n*